In [ ]:
for graph, label in zip([G, DHO_G], ['Original Data', 'DHO Predicted']):

    # Girvan-Newman community detection
    communities = nx.community.girvan_newman(graph)

    # Get the top level communities
    top_level_communities = next(communities)
    detected_communities = [list(c) for c in sorted(map(sorted, top_level_communities))]

    print(f"{label} number of communities: {len(detected_communities)}")


In [ ]:
# Compute the degree of each node (number of connections)
degrees = nx.degree(G)

# Sort nodes by degree in descending order
sorted_nodes_by_degree = sorted(degrees, key=lambda pair: pair[1], reverse=True)

# Get the top N nodes with the highest degree
N = 10  # For example, to get the top 5 most connected nodes
top_nodes_by_degree = sorted_nodes_by_degree[:N]

# Extract the node identifiers and their corresponding degrees
top_node_ids = [node_id for node_id, degree in top_nodes_by_degree]
top_node_degrees = [degree for node_id, degree in top_nodes_by_degree]

hubs = pd.DataFrame([(x,y) for x,y in zip(top_node_ids, top_node_degrees)], columns=['Node ID', 'Node Degree'])

# now get the hubs for DHO predicted network
degrees = nx.degree(DHO_G)

sorted_nodes_by_degree = sorted(degrees, key=lambda pair: pair[1], reverse=True)
top_nodes_by_degree = sorted_nodes_by_degree[:N]

# Extract the node identifiers and their corresponding degrees
top_node_ids = [node_id for node_id, degree in top_nodes_by_degree]
top_node_degrees = [degree for node_id, degree in top_nodes_by_degree]

hubs['DHO Node ID'] = top_node_ids
hubs['DHO Node Degree'] = top_node_degrees

hubs.to_csv('../data/1ES_compare_DHO_RN_hubs.csv', index=None)
hubs

In [ ]:
lilac = "#c994c7"
plt.figure()

for day in [418, 636]:
    plt.axvline(day, color='grey', alpha=0.7, linestyle=":")

node_degrees = sorted(nx.degree(DHO_G), key=lambda pair: pair[0], reverse=False)
degrees = [x[1] for x in node_degrees]
plt.plot(degrees, color=lilac, label='DHO', linewidth=1)

node_degrees = sorted(nx.degree(G), key=lambda pair: pair[0], reverse=False)
degrees = [x[1] for x in node_degrees]
plt.plot(degrees, color='k', label='1ES', zorder=2, linewidth=1)

plt.legend()
plt.xlabel(r't_{rest} (days)')
plt.ylabel('Node Degree')
plt.show()

In [ ]:
from statistics import *
get_graph_measures(G)


In [ ]:
# modularity -- a measure of the strength of the division of a network into communities
# High modularity means dense connections between the nodes within communities but sparse connections between nodes in different communities.
modularity = community_louvain.modularity(partition, G)
print(f"Overall Modularity of the Network: {modularity}")

centrality = nx.betweenness_centrality(G)
print(f"Community {i}: Centrality {centrality}")

# inter-community edges (number of edges that connect nodes of one community to another can be insightfu)
inter_community_edges = sum(1 for edge in G.edges() if partition[edge[0]] != partition[edge[1]])
print(f"Number of Inter-Community Edges: {inter_community_edges}")

total_edges = G.number_of_edges()
print(f"The total number of edges in the visibility graph: {total_edges}")

print("\n\nDHO PRED")
modularity = community_louvain.modularity(dho_partition, DHO_G)
print(f"Overall Modularity of the Network: {modularity}")

centrality = nx.betweenness_centrality(DHO_G)
print(f"Community: Average Centrality {np.mean(list(centrality.values()))}")

inter_community_edges = sum(1 for edge in DHO_G.edges() if dho_partition[edge[0]] != dho_partition[edge[1]])
print(f"Number of Inter-Community Edges: {inter_community_edges}")

total_edges = DHO_G.number_of_edges()
print(f"The total number of edges in the visibility graph: {total_edges}")



In [ ]:
nr, nr_rd = nx.non_randomness(G)
print(nr, nr_rd)

dho_nr, dho_nr_rd = nx.non_randomness(DHO_G)
print(dho_nr, dho_nr_rd)

In [ ]:
# Compute global efficiency
global_efficiency = nx.global_efficiency(G)

# Compute local efficiency for each node and then take the average for network's local efficiency
local_efficiency = nx.local_efficiency(G)

print("Original data:")
print("Global efficiency:", global_efficiency)
print("Local efficiency:", local_efficiency)

# Compute global efficiency
global_efficiency = nx.global_efficiency(DHO_G)

# Compute local efficiency for each node and then take the average for network's local efficiency
local_efficiency = nx.local_efficiency(DHO_G)

print("DHO pred:")
print("Global efficiency:", global_efficiency)
print("Local efficiency:", local_efficiency)

In [ ]:
# Compute graph measures for each graph
graph_measures_original = compute_basic_graph_measures(G)
graph_measures_dho = compute_basic_graph_measures(DHO_G)

graph_measure_names = list(graph_measures_original.keys())

data = {
    'Graph Measure': graph_measure_names,
    'Original': [graph_measures_original[measure] for measure in graph_measure_names],
    'DHO': [graph_measures_dho[measure] for measure in graph_measure_names]
}

# Create the DataFrame
df_graph_measures = pd.DataFrame(data)
print(df_graph_measures)


In [ ]:
# compute windowed RN measures

# # binned_ts = np.log10(binned_df['lumin'].values)
# binned_ts = y
# window_size = int(0.1*len(binned_ts))
# step_size = int(0.1*window_size)
# network_measures_df = compute_measures_in_parallel(binned_ts, window_size, step_size)
# print(network_measures_df)

In [ ]:
print("original data:")
compare_communities(nxg)

print("\n\n DHO pred:")
compare_communities(dho_nxg)

In [ ]:

community_transition_matrices = create_community_transition_matrices(communities_modularity)
global_transition_matrix = create_transition_matrix(G)
plot_diff_transition_matrix(community_transition_matrices, global_transition_matrix)

In [ ]:
degrees = nx.degree(G)

# degrees is a DegreeView object
degree_dict = dict(degrees)

# Extract degrees from the degree_dict
degrees = [degree for node, degree in degree_dict.items()]

# Plot histogram
plt.hist(degrees, bins=range(min(degrees), max(degrees) + 2))
plt.title('Histogram of Node Degrees')
plt.xlabel('Degree')
plt.ylabel('Number of Nodes')
plt.show()